In [60]:
import numpy as np
import pandas as pd
import sys, re
import os
import Jiyun.simple_index_builder as build
import Jiyun.simple_retrieval as retr
import json
import jsonpickle

In [9]:
DataFrame.columns.values.tolist()

'怎么推销'

In [118]:
## build index(inverted and forward index)
files = os.listdir("./Jiyun/corpus/")

for file in files:
    if "xlsx" in file:
        print("processing" + file)
        path = os.path.join("Jiyun/corpus/", file)
        qa_pairs = build.make_document_multi_round(path)
        build.build_index(qa_pairs)

processing达尔文语料分类-多类 Final (by 6.1).xlsx


In [119]:
## import index
inverted_index = None
forward_index = None
index1 = json.loads(open('inverted_index.json').read())
inverted_index = jsonpickle.decode(index1)
with open('forward_index.json', 'r') as fp:
    forward_index = json.load(fp)

In [202]:
## import test query
test_question = pd.read_csv('test_set.txt')
## 提取指定列
title_question = test_question.columns.values.tolist()[0]

query = []
for i in range(len(test_question)):
    query.append(test_question[title][i])  




In [203]:
## import ground truth
ground_truth1 = pd.read_csv('ground_truth1.txt')
ground_truth2 = pd.read_csv('ground_truth2.txt')
ground_truth3 = pd.read_csv('ground_truth3.txt')
ground_truth = [ground_truth1, ground_truth2, ground_truth3]
title_ground_truth = ground_truth1.columns.values.tolist()[0]

In [205]:
print(ground_truth1[title_ground_truth][0])
print(ground_truth2['问句'][1])
print(ground_truth3['问句'][1])


我想推销我们公司的购物卡，应该怎么推销呢？
你好呀 你是我的新朋友吗？
你好啊 你可以和我一起玩吗？


In [308]:
res_tmp = []
for i in range(len(query)):
    rslt = retr.retrieval(query[i], inverted_index)
    res_tmp.append(rslt)

Query is:
 推销 怎么
Query is:
 你好 达尔 达尔文
Query is:
 如何 购买 你
Query is:
 买 怎么 你
Query is:
 哪 在 买 你
Query is:
 网站 有 吗 你
Query is:
 APP 有 吗 你
Query is:
 下自 一下 介绍 自己 能 吗 你
Query is:
 功能 有 什么 你
Query is:
 陪 聊天 怎么 我
Query is:
 达尔 达尔文 智能 很 吗
Query is:
 赚钱 怎么
Query is:
 天 聊 我们 来 会儿
Query is:
 模式 训练 说 的 你
Query is:
 模式 说 聊天 的 你
Query is:
 初次 初次见面 见面 请多关照 关照
Query is:
 烦恼 很多 有 我
Query is:
 大数 数据 是 什么
Query is:
 意义 是 什么 的 生命
Query is:
 数据库 据库 数据 是 什么
Query is:
 虚拟 币 是 什么
Query is:
 谁 是 你
Query is:
 数字 货币 是 什么
Query is:
 人工 人工智能 智能 是 什么
Query is:
 免费 有 吗 你 免费版 版本
Query is:
 骗 钱 来 是 吗 的 我 你
Query is:
 信任 能 怎么 我 你
Query is:
 想 推销 我 面膜
Query is:
 一直 说 在 怎么 你 泰国
Query is:
 炒 币 吗 你
Query is:
 呆 着 今天 在家
Query is:
 思考 人生 会 吗 你
Query is:
 生活 是 什么 的 你 收入 来源
Query is:
 总是 为什么 这么 什么 你 积极 积极向上 向上
Query is:
 最 喜欢 是 什么 的 你 季节
Query is:
 总是 骗 觉得 在 我 你
Query is:
 觉得 啊 我 好累
Query is:
 总是 无关 一些 说 的 你 要紧
Query is:
 为了 工作 觉得 是 什么 你
Query is:
 不开 开心 有点 我
Query is:
 人生 在于 意义 呢 什么 的
Query is:
 兴趣 有点 对 我 你
Query is:
 好烦 真的 你

In [309]:
res_tmp[1][1].to_string()
forward_index[str(res_tmp[1][1].doc_id)]
str(res_tmp[1][1].confident)


'1.0'

In [251]:
forward_index[str(res_tmp[2][1].doc_id)].split('   ')[0].strip()

'你是如何选择现在这个推销事业的？'

In [312]:
res_tmp[1][1].to_string()

'2184 16.662825113315947'

In [314]:
test_question_res = []
for i in range(len(res_tmp)):
    count = 0
    model_res = []
    ## 暂时设定只有前三句话 top前三
    for j in range(3):
                # id & socre       # Q & A -> Q                                         # confident score   #labe
        rank = [res_tmp[i][j].to_string(), forward_index[str(res_tmp[i][j].doc_id)].split('   ')[0].strip(), str(res_tmp[i][j].confident), 0]
        #print(line.to_string() + ' ' + forward_index[str(line.doc_id)] + ' ' + str(line.confident) )
        ##print(forward_index[str(line.doc_id)])
        model_res.append(rank)
    test_question_res.append(model_res)                  

In [254]:
#ground_truth[0][title_ground_truth]


In [316]:
## initialize label using ground truth
for i in range(len(test_question_res)):
    for j in range(len(test_question_res[0])):
        res = test_question_res[i][j][1]
        if res == ground_truth[0][title_ground_truth][i] or res == ground_truth[1][title_ground_truth][i] or res == ground_truth[2][title_ground_truth][i]:
            test_question_res[i][j][3] = 1.0

导入数据，format： 
id    model_score   question   answer   label

In [3]:
## 每个问句都是返回前三sore的问答句pair，label暂时都为0，方便之后处理
pada1 = pd.read_csv('test1.txt', sep='\t')
pada2 = pd.read_csv('test2.txt', sep='\t')
pada3 = pd.read_csv('test3.txt', sep='\t')
dicts = [pada1, pada2, pada3]
##raters = pd.read_csv('raters.txt', sep='\t')

dicts = []  ### dicts[pada0, pada1, pada2, pada3.......padak]

In [188]:
def labeled(dicts, raters):
    tmp = dicts
    rater = raters.values.tolist()
    if tmp == None:
        print("Please load your test set!")
        return 0
    lens = 0
    if len(tmp) != len(raters):
        lens = len(tmp)
    else:
        lens = len(raters)
    
    for i in range(lens):
        for j in range(len(tmp[i]['answer'])):
            if tmp[i]['answer'][j] == rater[i][1]:
                tmp[i]['label'][j] = 1
                continue
            elif tmp[i]['answer'][j] == rater[i][2]:
                tmp[i]['label'][j] = 1
                continue
            elif tmp[i]['answer'][j] == rater[i][3]:
                tmp[i]['label'][j] = 1
                continue  
            else:
                tmp[i]['label'][j] = 0
    return tmp, rater

In [260]:
#test_question_res

In [315]:
len(test_question_res[0])


3

calculate the matrix
df.values.tolist()

In [324]:
sum = 0
### precision@1
for i in range(len(test_question_res)):
    #print(test_question_res[i][1][3])
    sum += test_question_res[i][0][3]
    
precision1 = sum / len(test_question_res)



In [325]:
precision1

0.15584415584415584

In [326]:
def precision(dicts, k):
    if dicts == None:
        print("Please load your test set!")
        return 0
    if k <= 0 or k > len(dicts):
        print("please input a valid value for k, which is from 1  to" + len(dicts))
        return 0
    
    sum = 0
    lens = len(dicts)
    ## 对于sample中每一个rank为k的问答句pair precision@k = sum / sample中所有rank为k的问句总数
    for i in range(lens):
        #print(test_question_res[i][1][3])
        sum += dicts[i][k-1][3]

    precision = sum / len(test_question_res)
    return precision




## MAP value
def mAP(dicts, k):
    
    if dicts == None:
        print("Please load your test set!")
        return 0
    if k <= 0 or k > len(dicts):
        print("please input a valid value for k, which is from 1  to" + len(dicts))
        return 0
    
    sum1 = 0
    for i in range(len(dicts)):
        sum2 = 0
        Nq = 0
        for j in range(k):
            sum2 += precision(dicts, j+1) * (precision(dicts, j+1) * len(dicts))
        for r in range(len(dicts[0])):
            Nq += dicts[i][r][3]

        if Nq == 0:
            sum1 += 0
        else:
            sum1 += sum2 / Nq

    return sum1 / len(dicts)


## MRR
def mRR(dicts):
    if dicts == None:
        print("Please load your test set!")
        return 0
    
    sum1 = 0
    for i in range(len(dicts)):
        for j in range(len(dicts[0])):
            if(dicts[i][j][3] == 1):
                sum1 += 1 / (j + 1)

    return sum1 / len(dicts)


## Z is a constant number
Z = 2.331

## nDCG
def nDCG(dicts, k):
    if dicts == None:
        print("Please load your test set!")
        return 0
    if k <= 0 or k > len(dicts):
        print("please input a valid value for k, which is from 1  to" + len(dicts))
        return 0
    
    sum1 = 0
    for i in range(len(dicts)):
        sum2 = 0
        for i in range(k):
            sum2 += (2**(precision(dicts, i+1) * len(dicts)) - 1) / np.log(1 + (i+1))
        
        sum1 += sum2 / Z
                     
        return sum1 / len(dicts)


In [331]:
print("precision@1 is: " + str(precision(test_question_res, 1)))
print("MAP@1 is: " + str(mAP(test_question_res, 1)))
print("MRR is: " + str(mRR(test_question_res)))
print("nDCG@1 is: " + str(nDCG(test_question_res, 1)))

precision@1 is: 0.15584415584415584
MAP@1 is: 0.3764547141170518
MRR is: 0.20346320346320346
nDCG@1 is: 32.915120273001975


In [332]:
print("precision@1 is: " + str(precision(test_question_res, 1)))
print("precision@2 is: " + str(precision(test_question_res, 2)))
print("precision@3 is: " + str(precision(test_question_res, 3)))

precision@1 is: 0.15584415584415584
precision@2 is: 0.07792207792207792
precision@3 is: 0.025974025974025976
